# Exact Diagonalization using XDiag and Julia

> **Alexander Wietek**
>
> Max Planck Institute for the Physics of Complex Systems, Dresden
>
> https://awietek.github.io/xdiag/
>
> https://www.pks.mpg.de/smc

## Creating Hilbert spaces
First we load the XDiag package for Exact Diagonalization

In [ ]:
using XDiag

Let us start with a single spin $S=1/2$ particle and create its Hilbert space, and print out the configurations.

In [ ]:
hspace = Spinhalf(1)
for s in hspace
    print(s)
end

We can so the same for 2-particles, 

In [ ]:
hspace = Spinhalf(2)
for s in hspace
    print(s)
end

3-particles, and so on.

In [ ]:
hspace = Spinhalf(3)
for s in hspace
    print(s)
end

## Defining operators

We define our first operator acting on two spin one-half particles. We define the Heisenberg ("HB") interaction given by

$$ \mathbf{S}_i\cdot \mathbf{S}_j = S^x_i S^x_j + S^y_i S^y_j + S^z_i S^z_j$$

where the spin matrices $S^\alpha$ are given by

$$ S^x = \frac{1}{2} \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \quad
S^y = \frac{1}{2} \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix} \quad
S^z = \frac{1}{2} \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}
$$

To do so, we create an "OpSum" object, which encodes the sum of local operators, and add an "Op" object to it describing the Heisenberg interaction.

In [ ]:
ops = OpSum()
ops += Op("HB", 1.0, [1, 2])
hspace = Spinhalf(2)
H = matrix(ops, hspace)

For the "Op" object the first argument defines the type of the interaction, in this case the Heisenberg interaction "HB". The second arguments sets the prefactor, in this case just 1.0. The third argument is a list of the sites this operator is acting on. Now we are interested in the eigenvalues and eigenvectors of this interaction.

In [ ]:
using LinearAlgebra
eigen(H)

We see there is one eigenvalue $\varepsilon_1 = -3/4$ and three degenerate eigenvalues $\varepsilon_{2,3,4} = 1/4$. The corresponding eigenvectors are:

$$
\begin{align}
| \psi_1 \rangle &= \frac{1}{\sqrt{2}}(|\uparrow\downarrow\rangle - |\downarrow\uparrow\rangle) \\
| \psi_2 \rangle &= |\downarrow\downarrow\rangle \\
| \psi_3 \rangle &= \frac{1}{\sqrt{2}}(|\uparrow\downarrow\rangle + |\downarrow\uparrow\rangle) \\
| \psi_4 \rangle &= |\uparrow\uparrow\rangle 
\end{align}
$$

So we see the ground state is the singlet state with total spin $S=0$ and the excited states are the triplet states with $S=1$.

### Exersise 1:

Create a Hilbert space on three sites denoting the edges 1,2, and 3 of a triangle. Then consider the Heisenberg model on this triangle, i.e. Heisenberg interactions between the site pairs [1, 2], [2, 3], and [3, 1]. Compute the eigenvalues and eigenvectors, to find out what is the degeneracy and total spin of the ground state and the excited states. Can all degeneracies be explained by spin rotation $SU(2)$ symmetry?

#### Solution

In [ ]:
hspace = Spinhalf(3)
ops = OpSum()
ops += Op("HB", 1.0, [1, 2])
ops += Op("HB", 1.0, [2, 3])
ops += Op("HB", 1.0, [3, 1])
H = matrix(ops, hspace)
eigen(H)

## $S^z$ conservation

The Heisenberg interaction conserved the total $S^z$. Thus, our Hilbert space decomposes into blocks which are not coupled by one another, where configurations have a fixed value of the total $S^z$. We can create these blocks and express the H

In [ ]:
ops = OpSum()
ops += Op("HB", 1.0, [1, 2])
ops += Op("HB", 1.0, [2, 3])
ops += Op("HB", 1.0, [3, 1])
for nup in 0:3
    block = Spinhalf(3, nup)
    H = matrix(ops, block)
    display(H)
end

## Translational symmetry

Let us refer to the Hamiltonian,

$$ H = \sum_{i=1}^N \mathbf{S}_i\cdot \mathbf{S}_{(i+1) \textrm{ mod } N} $$

to the $N$-site Heisenberg chain with periodic boundary conditions. Hamiltonians can be invariant under translation symmetries. In the example with the three-site Heisenberg model, the Hamiltonian is invariant under exchanging the sites $1 \rightarrow 2 \rightarrow 3 \rightarrow 1$. More formally, we can define a translation operator by

$$ T: |\sigma_1 \sigma_2 \cdots \sigma_{N-1} \sigma_N \rangle \rightarrow |\sigma_N \sigma_1 \cdots \sigma_{N-2} \sigma_{N-1} \rangle$$ 

A symmetry $S$ of a Hamiltonian $H$ is an operator which commutes with $H$, i.e.

$$ [H, S] \equiv H S - S H = 0$$

For a cyclic Hamiltonian, one can easily check that $T$ commutes with $H$. If $S$ and $T$ commute with a Hamiltonian, so does $ST$,

$$ HST = SHT = STH \Rightarrow [H, ST] = 0$$

Similarly, one can also see that the inverse of $S$ needs to be yet another symmetry. Hence, the set of symmetries forms a mathematical group, the symmetry group. A key concept in quantum mechanics is that every symmetry group divides the Hilbert space into **blocks** which can be labeled by a so-called **quantum number** $\rho$. Quantum numbers refer to the eigenvalues of the symmetry operator. Assume we have a state $|\psi\rangle$ which fulfills

$$ S |\psi_\rho\rangle = \rho |\psi_\rho\rangle $$

Then, also $H |\psi_\rho\rangle$ is an eigenstate of $S$ with eigenvector $\rho$ due to, 

$$ S (H |\psi_\rho\rangle) = H S|\psi_\rho\rangle = \rho (H|\psi_\rho\rangle).$$

Now explicitly for an $N$-site Heisenberg chain, given a spin configuration $|\sigma_1 \sigma_2 \cdots \sigma_{N-1} \sigma_N \rangle$ we can consider the so-called **symmetry-adapted state**,

$$ P_\rho |\sigma_1 \sigma_2 \cdots \sigma_{N-1} \sigma_N \rangle= \sum_{n=0}^{N-1} e^{i k n} T^n |\sigma_1 \sigma_2 \cdots \sigma_{N-1} \sigma_N \rangle. $$

We have,

$$ T P_\rho |\sigma_1 \sigma_2 \cdots \sigma_{N-1} \sigma_N \rangle = e^{-i k} P_\rho |\sigma_1 \sigma_2 \cdots \sigma_{N-1}\rangle$$

and, hence, $P_\rho |\sigma_1 \sigma_2 \cdots \sigma_{N-1} \sigma_N \rangle$ is an eigenstate of $T$. Since $T^n = \textrm{Id}$, the values of $k$ need to satisfy $k = 2\pi l / n$, where $l$ is an integer. The Hamiltonian can now be expressed in the basis of these symmetry adapted states, where it then decomposes into blocks labeled by there quantum number, in this case the **(quasi-)momentum**. More generically, the numbers $\chi(n) = e^{i k n}$ can have different forms for different symmetry groups and are refered to as an (one-dimensional) **irreducible representation (irrep)** of the symmetry group.

In [ ]:
N = 4
nup = 2
ops = OpSum()
for i in 1:N
    ops += Op("HB", 1.0, [i, mod1(i+1, N)])
end

id = Permutation([1, 2, 3, 4])
T = Permutation([2, 3, 4, 1])
group = PermutationGroup([id, T, T*T, T*T*T])

# 0-momentum irrep
irrep = Representation([1, 1, 1, 1])
block = Spinhalf(N, nup, group, irrep)
for s in block
    print(s)
end
println()

H = matrix(ops, block)
println("0 momentum")
display(H)
display(eigvals(H))

# pi/2-momentum irrep
irrep = Representation([1, 1im, -1, -1im])
block = Spinhalf(N, nup, group, irrep)
H = matrix(ops, block)
println("pi/2 momentum")
display(H)
display(eigvals(Hermitian(H)))

# pi-momentum irrep
irrep = Representation([1, -1, 1, -1])
block = Spinhalf(N, nup, group, irrep)
H = matrix(ops, block)
println("pi momentum")
display(H)
display(eigvals(H))

### Exersize 2

Compute the full spectrum of the four-site Heisenberg chain, both with and without momentum conservation. Check whether the results from both computations agree.

#### Solution

In [ ]:
N = 4
ops = OpSum()
for i in 1:N
    ops += Op("HB", 1.0, [i, mod1(i+1, N)])
end
block = Spinhalf(N)
H = matrix(ops, block)
eigs = eigvals(Symmetric(H))

id = Permutation([1, 2, 3, 4])
T = Permutation([2, 3, 4, 1])
group = PermutationGroup([id, T, T*T, T*T*T])

ir1 = Representation([1, 1, 1, 1]) 
ir2 = Representation([1, -1, 1, -1])    
ir3 = Representation([1, im, -1, -im])    
ir4 = Representation([1, -im, -1, im])    

eigs_sym = []
for nup in 0:N
    for irrep in [ir1, ir2, ir3, ir4]
        block = Spinhalf(N, nup, group, irrep)
        H = matrix(ops, block)
        append!(eigs_sym, eigvals(Hermitian(H)))
    end
end
sort!(eigs_sym)
display(eigs)
display(eigs_sym)
@show isapprox(eigs, eigs_sym)

## Sparse Diagonalization and Algebra

Instead of dealing with the full matrix of a Hamiltonian, XDiag implements iterative algorithms such as the [Lanczos algorithm](https://en.wikipedia.org/wiki/Lanczos_algorithm). It can be used to perform ED on larger system sizes. Moreover, there are several functions to apply operators and create zero or random states.

In [ ]:
N = 10
ops = OpSum()
for i in 1:N
    ops += Op("HB", 1.0, [i, mod1(i+1, N)])
end
block = Spinhalf(N)
H = matrix(ops, block)
eigs = eigvals(Symmetric(H))

e0, gs = eig0(ops, block)

Hgs = zeros(block)
apply(ops, gs, Hgs)
@show e0, eigs[1]
@show norm(gs)
@show dot(gs, Hgs)
@show inner(ops, gs)

rstate = rand(block)
@show inner(ops, rstate)

### Exercise 3

Implement the Lanczos algorithm using the functions **rand**, and **apply** above to compute the ground state energy of a 10-site Heisenberg chain. 

#### Solution

In [ ]:
N = 10
ops = OpSum()
for i in 1:N
    ops += Op("HB", 1.0, [i, mod1(i+1, N)])
end
block = Spinhalf(N)
    
function lanczos(ops, block; precision=1e-12, max_iterations=1000)
    
    alphas = Float64[]
    betas = Float64[]

    v1 = rand(block; normalized=true)
    v0 = zeros(block)
    w = zeros(block)

    alpha = 0.0
    beta = 0.0
        
    prev_energy = 0
    for iteration in 1:max_iterations

        # Perform basic Lanczos iteration step
        apply(ops, v1, w)
        alpha = dot(v1, w)
        w = w - alpha * v1 - beta * v0
        v0 = v1

        beta = norm(w)
        v1 = w / beta

        push!(alphas, alpha)

        # Build up T-matrix
        t_matrix = diagm(0 => alphas, 1 => betas, -1 => betas)
        t_eigs = eigvals(Symmetric(t_matrix))

        push!(betas, beta)
        @show iteration, t_eigs[1]

        # Return if converged
        if isapprox(t_eigs[1], prev_energy; rtol=precision)
            println("Lanczos converged in $iteration steps")
            return t_eigs, alphas, betas
        end
        prev_energy = t_eigs[1]
    end            
end

t_eigs, alphas, betas = lanczos(ops, block)
@show t_eigs[1]
@show eigval0(ops, block)

## Convergence of the Lanczos algorithm

In [ ]:
using CairoMakie
f = Figure()
ax = Axis(f[1, 1])
niter = length(alphas)
for iter in 1:niter
    t_matrix = diagm(0 => alphas[1:iter], 1 => betas[1:iter-1], -1 => betas[1:iter-1])
    t_eigs = eigvals(Symmetric(t_matrix))
    scatter!(ax, iter*ones(size(t_eigs)), t_eigs)
end
ax.xlabel = "iteration"
ax.ylabel = "eigenvalue"
display(f)

## Ground state correlations of triangular lattice Heisenberg model

In [ ]:
using TOML
N = 12
latfile = TOML.parsefile("triangular.$N.J1J2.toml")
    
coords = latfile["Coordinates"]
interactions = latfile["Interactions"]
ops = OpSum()
for interaction in interactions
    type = interaction[1]
    couplingname = interaction[2]
    s1 = interaction[3] + 1
    s2 = interaction[4] + 1
    ops += Op(type, couplingname, [s1, s2])
end
ops["J1"] = 1.0
ops["J2"] = 0.0

block = Spinhalf(N, N÷2)
e0, gs = eig0(ops, block)
@show e0
    
for i in 2:N
    s1si = Op("HB", 1.0, [1, i])
    corr = inner(s1si, gs)
    println("$i $corr")
end

### Exercise 4

Plot the correlation function on the triangular lattice defined by its coordinates, as read in from the text file above. What kind of magnetic order does the ground state have?

## Ground state correlations of triangular lattice Heisenberg model (with translational symmetries)

In [ ]:
    permutation_vecs = latfile["Symmetries"]
    permutations = Permutation[]
    for permutation_vec in permutation_vecs
        push!(permutations, Permutation(permutation_vec .+ 1))
    end
    group = PermutationGroup(permutations)
    irrep = Representation(ones(size(group)))
    block = Spinhalf(N, N÷2, group, irrep)
    e0, gs = eig0(ops, block)
    @show e0

    for i in 2:N
        s1si = symmetrize(Op("HB", 1.0, [1, i]), group)
        corr = inner(s1si, gs)
        println("$i $corr")
    end